# Object tracking & motion analysis

## فیلتر کردن رنگ

### فیلتر بر اساس Hue (فام)

- بازه رنگ Hue از ۰ تا ۱۸۰ است

- بازه‌های فیلتر رنگ:
    - قرمز: ۱۶۵ تا ۱۵
    - سبز: ۴۵ تا ۷۵
    - آبی: ۹۰ تا ۱۲۰

### فیلتر بر اساس اشباع (Saturation) و روشنایی (Value/Brightness)

- معمولاً بازه فیلتر را برای اشباع و روشنایی از ۵۰ تا ۲۵۵ قرار می‌دهیم تا رنگ‌های واقعی را پوشش دهد
- بازه ۰ تا ۶۰ در اشباع نزدیک به سفید است
- بازه ۰ تا ۶۰ در روشنایی نزدیک به سیاه است


## Filter by color

In [ ]:
import cv2
import numpy as np

# Initialize webcam
cap = cv2.VideoCapture(0)

# define range of PURPLE color in HSV
lower_purple = np.array([125,0,0])
upper_purple = np.array([175,255,255])

# loop until break statement is exectured
while True:
    
    # Read webcam image
    ret, frame = cap.read()
    
    # Convert image from RBG/BGR to HSV so we easily filter
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)


    # Use inRange to capture only the values between lower & upper_blue
    mask = cv2.inRange(hsv_img, lower_blue, upper_blue)

    # Perform Bitwise AND on mask and our original frame
    res = cv2.bitwise_and(frame, frame, mask=mask)

    cv2.imshow('Original', frame)  
    cv2.imshow('mask', mask)
    cv2.imshow('Filtered Color Only', res)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()

این کد برای فیلتر کردن یک رنگ خاص (در اینجا بنفش) از تصویر دریافتی وب‌کم استفاده می‌کند. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`cv2` و `numpy`) وارد می‌شوند.
- وب‌کم با استفاده از `cv2.VideoCapture(0)` فعال می‌شود.
- محدوده رنگ بنفش در فضای رنگی HSV با دو آرایه `lower_purple` و `upper_purple` تعریف می‌شود.
- یک حلقه بی‌نهایت اجرا می‌شود تا زمانی که کلید Enter فشرده شود.
- در هر تکرار حلقه:
    - یک فریم از وب‌کم خوانده می‌شود.
    - تصویر از فضای رنگی BGR به HSV تبدیل می‌شود تا فیلتر کردن رنگ راحت‌تر انجام شود.
    - با استفاده از تابع `cv2.inRange`، ماسکی ساخته می‌شود که فقط پیکسل‌هایی که در محدوده رنگ بنفش قرار دارند را نگه می‌دارد.
    - با استفاده از عملگر بیت‌به‌بیت AND، فقط بخش‌هایی از تصویر که رنگ بنفش دارند نمایش داده می‌شوند.
    - سه پنجره نمایش داده می‌شود: تصویر اصلی، ماسک، و تصویر فیلتر شده.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، وب‌کم آزاد شده و تمام پنجره‌ها بسته می‌شوند.

**نکته:** در کد نمونه، متغیرهای `lower_blue` و `upper_blue` به اشتباه استفاده شده‌اند و باید به `lower_purple` و `upper_purple` تغییر داده شوند.


## Background Subtraction

این یک تکنیک بسیار مفید در بینایی ماشین است که به ما اجازه می‌دهد اجسام متحرک (پیش‌زمینه) را از پس‌زمینه در یک جریان ویدیویی جدا کنیم.

این الگوریتم‌ها اساساً درباره فریم موجود در تصویر (جریان ویدیو) یاد می‌گیرند و می‌توانند به‌طور دقیق ماسک پیش‌زمینه را شناسایی کنند. نتیجه این کار، یک تقسیم‌بندی دودویی از تصویر است که نواحی اجسام غیرثابت را برجسته می‌کند.

در OpenCV چندین الگوریتم تفریق پس‌زمینه مخصوص تحلیل ویدیو وجود دارد:

- **BackgroundSubtractorMOG** – یک الگوریتم جداسازی پیش‌زمینه/پس‌زمینه مبتنی بر مخلوط گاوسی. هر پیکسل پس‌زمینه را با یک مخلوطی از توزیع‌های گاوسی مدل می‌کند و با یک تقریب آنلاین مدل را به‌روزرسانی می‌کند.
- **BackgroundSubtractorMOG2** – نسخه بهبود یافته MOG که سازگاری بهتری با صحنه‌های متغیر و سایه‌ها دارد.
- **BackgroundSubtractorGMG** – از تخمین آماری تصویر پس‌زمینه و تقسیم‌بندی بیزی پیکسل‌به‌پیکسل استفاده می‌کند.

این الگوریتم‌ها به طور گسترده در کاربردهایی مانند ردیابی اشیا، تشخیص حرکت و نظارت تصویری استفاده می‌شوند.


### 1. Gaussian Mixture-based Background/Foreground Segmentation Algorithm

In [ ]:
# OpenCV 2.4.13 only
import numpy as np
import cv2

cap = cv2.VideoCapture('walking.avi')

# Initlaize background subtractor
foreground_background = cv2.BackgroundSubtractorMOG()

while True:
    
    ret, frame = cap.read()

    # Apply background subtractor to get our foreground mask
    foreground_mask = foreground_background.apply(frame)

    cv2.imshow('Output', foreground_mask)
    if cv2.waitKey(1) == 13: 
        break

cap.release()
cv2.destroyAllWindows()

این کد برای جداسازی پیش‌زمینه و پس‌زمینه در یک ویدیو با استفاده از الگوریتم مخلوط گاوسی (MOG) در OpenCV نسخه 2.4.13 نوشته شده است. توضیح مراحل به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- ویدیوی `'walking.avi'` با استفاده از `cv2.VideoCapture` بارگذاری می‌شود.
- یک شیء از کلاس `BackgroundSubtractorMOG` ساخته می‌شود تا مدل پس‌زمینه را ایجاد کند.
- یک حلقه بی‌نهایت اجرا می‌شود:
    - هر بار یک فریم از ویدیو خوانده می‌شود.
    - با استفاده از متد `apply`، ماسک پیش‌زمینه استخراج می‌شود؛ یعنی بخش‌هایی از تصویر که متحرک هستند جدا می‌شوند.
    - ماسک به دست آمده نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، ویدیو آزاد شده و تمام پنجره‌ها بسته می‌شوند.

این روش برای تشخیص حرکت و ردیابی اشیاء متحرک در ویدیو کاربرد دارد.


### اگر بخواهیم این روش را روی ورودی وب‌کم خود اجرا کنیم چه؟

In [ ]:
# OpenCV 2.4.13 only
import numpy as np
import cv2

# Intialize Webcam
cap = cv2.VideoCapture(0)

# Initlaize background subtractor
foreground_background = cv2.BackgroundSubtractorMOG()

while True:
    
    ret, frame = cap.read()

    # Apply background subtractor to get our foreground mask
    foreground_mask = foreground_background.apply(frame)

    cv2.imshow('Output', foreground_mask)
    if cv2.waitKey(1) == 13: 
        break

cap.release()
cv2.destroyAllWindows()

این کد برای جداسازی پیش‌زمینه و پس‌زمینه در تصویر دریافتی از وب‌کم با استفاده از الگوریتم مخلوط گاوسی (MOG) در OpenCV نسخه 2.4.13 نوشته شده است. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- وب‌کم با استفاده از `cv2.VideoCapture(0)` فعال می‌شود.
- یک شیء از کلاس `BackgroundSubtractorMOG` ساخته می‌شود تا مدل پس‌زمینه را ایجاد کند.
- یک حلقه بی‌نهایت اجرا می‌شود:
    - هر بار یک فریم از وب‌کم خوانده می‌شود.
    - با استفاده از متد `apply`، ماسک پیش‌زمینه استخراج می‌شود؛ یعنی بخش‌هایی از تصویر که متحرک هستند جدا می‌شوند.
    - ماسک به دست آمده نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، وب‌کم آزاد شده و تمام پنجره‌ها بسته می‌شوند.

این روش برای تشخیص حرکت و ردیابی اشیاء متحرک در تصویر وب‌کم کاربرد دارد.


### بیایید مدل مخلوط گاوسی تطبیقی بهبود یافته را برای تفریق پس‌زمینه امتحان کنیم

In [ ]:
# OpenCV 2.4.13
import numpy as np
import cv2

cap = cv2.VideoCapture('walking.avi')

# Initlaize background subtractor
foreground_background = cv2.BackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()

    # Apply background subtractor to get our foreground mask
    foreground_mask = foreground_background.apply(frame)

    cv2.imshow('Output', foreground_mask)
    if cv2.waitKey(1) == 13: 
        break

cap.release()
cv2.destroyAllWindows()

این کد برای جداسازی پیش‌زمینه و پس‌زمینه در یک ویدیو با استفاده از الگوریتم مخلوط گاوسی تطبیقی (MOG2) در OpenCV نسخه 2.4.13 نوشته شده است. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- ویدیوی `'walking.avi'` با استفاده از `cv2.VideoCapture` بارگذاری می‌شود.
- یک شیء از کلاس `BackgroundSubtractorMOG2` ساخته می‌شود تا مدل پس‌زمینه را ایجاد کند.
- یک حلقه بی‌نهایت اجرا می‌شود:
    - هر بار یک فریم از ویدیو خوانده می‌شود.
    - با استفاده از متد `apply`، ماسک پیش‌زمینه استخراج می‌شود؛ یعنی بخش‌هایی از تصویر که متحرک هستند جدا می‌شوند.
    - ماسک به دست آمده نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، ویدیو آزاد شده و تمام پنجره‌ها بسته می‌شوند.

این روش برای تشخیص حرکت و ردیابی اشیاء متحرک در ویدیو کاربرد دارد و نسبت به نسخه قبلی (MOG) عملکرد بهتری در شرایط نوری متغیر و سایه‌ها دارد.


### اعمال آن بر روی جریان وب‌کم خود

In [ ]:
# OpenCV 2.4.13
import numpy as np
import cv2

# Intialize Webcam
cap = cv2.VideoCapture(0)

# Initlaize background subtractor
foreground_background = cv2.BackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    
    # Apply background subtractor to get our foreground mask
    foreground_mask = foreground_background.apply(frame)

    cv2.imshow('Output', foreground_mask)
    if cv2.waitKey(1) == 13: 
        break

cap.release()
cv2.destroyAllWindows()

این کد برای جداسازی پیش‌زمینه و پس‌زمینه در تصویر دریافتی از وب‌کم با استفاده از الگوریتم مخلوط گاوسی تطبیقی (MOG2) در OpenCV نسخه 2.4.13 نوشته شده است. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- وب‌کم با استفاده از `cv2.VideoCapture(0)` فعال می‌شود.
- یک شیء از کلاس `BackgroundSubtractorMOG2` ساخته می‌شود تا مدل پس‌زمینه را ایجاد کند.
- یک حلقه بی‌نهایت اجرا می‌شود:
    - هر بار یک فریم از وب‌کم خوانده می‌شود.
    - با استفاده از متد `apply`، ماسک پیش‌زمینه استخراج می‌شود؛ یعنی بخش‌هایی از تصویر که متحرک هستند جدا می‌شوند.
    - ماسک به دست آمده نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، وب‌کم آزاد شده و تمام پنجره‌ها بسته می‌شوند.

این روش برای تشخیص حرکت و ردیابی اشیاء متحرک در تصویر وب‌کم کاربرد دارد و نسبت به نسخه قبلی (MOG) عملکرد بهتری در شرایط نوری متغیر و سایه‌ها دارد.


## تفریق پیش‌زمینه چیست؟

In [ ]:
import cv2
import numpy as np

# Initalize webacam and store first frame
cap = cv2.VideoCapture(0)
ret, frame = cap.read()

# Create a flaot numpy array with frame values
average = np.float32(frame)

while True:
    # Get webcam frmae
    ret, frame = cap.read()
    
    # 0.01 is the weight of image, play around to see how it changes
    cv2.accumulateWeighted(frame, average, 0.01)
    
    # Scales, calculates absolute values, and converts the result to 8-bit
    background = cv2.convertScaleAbs(average)

    cv2.imshow('Input', frame)
    cv2.imshow('Disapearing Background', background)
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cv2.destroyAllWindows()
cap.release()

این کد برای حذف تدریجی پس‌زمینه از تصویر دریافتی وب‌کم با استفاده از میانگین‌گیری وزنی (Background Averaging) نوشته شده است. توضیح مراحل به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`cv2` و `numpy`) وارد می‌شوند.
- وب‌کم با استفاده از `cv2.VideoCapture(0)` فعال می‌شود و اولین فریم خوانده می‌شود.
- یک آرایه‌ی شناور (`float32`) با مقادیر اولین فریم ساخته می‌شود تا به عنوان میانگین اولیه پس‌زمینه استفاده شود.
- یک حلقه بی‌نهایت اجرا می‌شود:
    - در هر تکرار، یک فریم جدید از وب‌کم خوانده می‌شود.
    - با استفاده از تابع `cv2.accumulateWeighted`، میانگین وزنی بین فریم فعلی و میانگین قبلی محاسبه می‌شود. پارامتر ۰.۰۱ وزن فریم جدید را تعیین می‌کند (هرچه این عدد بزرگ‌تر باشد، پس‌زمینه سریع‌تر به‌روزرسانی می‌شود).
    - با استفاده از `cv2.convertScaleAbs`، میانگین به تصویر ۸ بیتی تبدیل می‌شود تا قابل نمایش باشد.
    - دو پنجره نمایش داده می‌شود: یکی تصویر ورودی و دیگری پس‌زمینه‌ی محو شونده.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، تمام پنجره‌ها بسته شده و وب‌کم آزاد می‌شود.

این روش برای حذف تدریجی اجسام متحرک و نمایش پس‌زمینه‌ی ثابت کاربرد دارد.


### Background Substraction KKN
#### OpenCV 3.X only!

In [ ]:
# OpenCV 3.1.0
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbg = cv2.createBackgroundSubtractorKNN()

while(1):
    ret, frame = cap.read()

    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    cv2.imshow('frame',fgmask)
    
    if cv2.waitKey(1) == 13: 
        break

cap.release()
cv2.destroyAllWindows()

این کد برای جداسازی پیش‌زمینه و پس‌زمینه در تصویر دریافتی از وب‌کم با استفاده از الگوریتم KNN (K-Nearest Neighbors) در OpenCV نسخه 3.1.0 نوشته شده است. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- وب‌کم با استفاده از `cv2.VideoCapture(0)` فعال می‌شود.
- یک کرنل بیضوی با اندازه ۳x۳ برای عملیات مورفولوژیکی ساخته می‌شود.
- شیء `createBackgroundSubtractorKNN` برای تفریق پس‌زمینه با الگوریتم KNN ساخته می‌شود.
- یک حلقه بی‌نهایت اجرا می‌شود:
    - هر بار یک فریم از وب‌کم خوانده می‌شود.
    - با استفاده از متد `apply`، ماسک پیش‌زمینه استخراج می‌شود؛ یعنی بخش‌هایی از تصویر که متحرک هستند جدا می‌شوند.
    - با استفاده از عملیات مورفولوژیکی `MORPH_OPEN`، نویزهای کوچک از ماسک حذف می‌شوند تا تصویر تمیزتری به دست آید.
    - ماسک نهایی نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، وب‌کم آزاد شده و تمام پنجره‌ها بسته می‌شوند.

این روش برای تشخیص حرکت و ردیابی اشیاء متحرک در تصویر وب‌کم کاربرد دارد و نسبت به روش‌های قبلی در برخی شرایط عملکرد بهتری دارد.


## Meanshift - An Object tracking algorithm (میان‌شیفت - یک الگوریتم ردیابی شیء)
این الگوریتم بر پایه یک ایده ساده بنا شده است: با یافتن بیشترین تراکم نقاط نمونه‌برداری شده و بازمحاسبه آن در فریم بعدی، پنجره مشاهده ما به سمت حرکت شیء جابجا می‌شود.

در OpenCV معمولاً از تصویر back projection هیستوگرام و موقعیت اولیه هدف برای ردیابی استفاده می‌شود.

In [ ]:
import numpy as np
import cv2

# Initialize webcam
cap = cv2.VideoCapture(0)

# take first frame of the video
ret, frame = cap.read()
print(type(frame))

# setup default location of window
r, h, c, w = 240, 100, 400, 160 
track_window = (c, r, w, h)

# Crop region of interest for tracking
roi = frame[r:r+h, c:c+w]

# Convert cropped window to HSV color space
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Create a mask between the HSV bounds
lower_purple = np.array([125,0,0])
upper_purple = np.array([175,255,255])
mask = cv2.inRange(hsv_roi, lower_purple, upper_purple)

# Obtain the color histogram of the ROI
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0,180])

# Normalize values to lie between the range 0, 255
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Setup the termination criteria
# We stop calculating the centroid shift after ten iterations 
# or if the centroid has moved at least 1 pixel
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    
    # Read webcam frame
    ret, frame = cap.read()

    if ret == True:
        
        # Convert to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Calculate the histogram back projection 
        # Each pixel's value is it's probability
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Draw it on image
        x, y, w, h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w, y+h), 255, 2)    

        cv2.imshow('Meansift Tracking', img2)
        
        if cv2.waitKey(1) == 13: #13 is the Enter Key
            break

    else:
        break

cv2.destroyAllWindows()
cap.release()

این کد یک پیاده‌سازی از الگوریتم میان‌شیفت (Meanshift) برای ردیابی شیء در تصویر دریافتی از وب‌کم است. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- وب‌کم با استفاده از `cv2.VideoCapture(0)` فعال می‌شود.
- اولین فریم از وب‌کم خوانده می‌شود تا ناحیه اولیه ردیابی انتخاب شود.
- مختصات و ابعاد پنجره ردیابی (`track_window`) تعیین می‌شود و ناحیه مورد نظر (ROI) از تصویر بریده می‌شود.
- ناحیه انتخابی به فضای رنگی HSV تبدیل می‌شود تا فیلتر رنگ راحت‌تر انجام شود.
- یک ماسک برای محدوده رنگ بنفش ایجاد می‌شود تا فقط پیکسل‌های این رنگ در ناحیه مورد نظر باقی بمانند.
- هیستوگرام رنگی ناحیه انتخابی محاسبه و نرمال‌سازی می‌شود تا برای ردیابی استفاده شود.
- معیار توقف الگوریتم میان‌شیفت تعیین می‌شود (۱۰ تکرار یا جابجایی کمتر از ۱ پیکسل).
- در یک حلقه بی‌نهایت:
    - هر بار یک فریم جدید از وب‌کم خوانده می‌شود.
    - تصویر به فضای HSV تبدیل می‌شود.
    - تصویر back projection هیستوگرام محاسبه می‌شود تا احتمال حضور شیء در هر نقطه مشخص شود.
    - الگوریتم میان‌شیفت روی تصویر back projection اجرا می‌شود تا موقعیت جدید پنجره ردیابی به دست آید.
    - یک مستطیل روی شیء ردیابی‌شده رسم و نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، تمام پنجره‌ها بسته شده و وب‌کم آزاد می‌شود.

این کد برای ردیابی اشیاء رنگی (در اینجا بنفش) در تصویر ویدیویی کاربرد دارد و با استفاده از الگوریتم میان‌شیفت موقعیت شیء را در هر فریم به‌روزرسانی می‌کند.


## Camshift - An object tracking Algorithm (کم‌شیفت - یک الگوریتم ردیابی شیء)
الگوریتم Camshift بسیار شبیه به الگوریتم Meanshift است، اما شاید متوجه شده باشید که پنجره در Meanshift اندازه ثابتی دارد. این موضوع می‌تواند مشکل‌ساز باشد، زیرا حرکت اشیا در تصویر می‌تواند کوچک یا بزرگ باشد و اگر پنجره بیش از حد بزرگ باشد، ممکن است شیء هنگام ردیابی از دست برود.

Camshift (مخفف continuously adaptive Meanshift) از یک پنجره تطبیقی استفاده می‌کند که اندازه و حتی جهت آن (یعنی می‌تواند بچرخد) تغییر می‌کند. مراحل ساده‌شده این الگوریتم به شرح زیر است:

1. اجرای Meanshift تا زمانی که همگرا شود
2. محاسبه اندازه پنجره
3. محاسبه جهت با استفاده از بهترین بیضی برازش‌یافته


In [ ]:
import numpy as np
import cv2

# Initialize webcam
cap = cv2.VideoCapture(0)

# take first frame of the video
ret, frame = cap.read()

# setup default location of window
r, h, c, w = 240, 100, 400, 160 
track_window = (c, r, w, h)

# Crop region of interest for tracking
roi = frame[r:r+h, c:c+w]

# Convert cropped window to HSV color space
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Create a mask between the HSV bounds
lower_purple = np.array([130,60,60])
upper_purple = np.array([175,255,255])
mask = cv2.inRange(hsv_roi, lower_purple, upper_purple)

# Obtain the color histogram of the ROI
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0,180])

# Normalize values to lie between the range 0, 255
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Setup the termination criteria
# We stop calculating the centroid shift after ten iterations 
# or if the centroid has moved at least 1 pixel
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    
    # Read webcam frame
    ret, frame = cap.read()

    if ret == True:
        # Convert to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Calculate the histogram back projection 
        # Each pixel's value is it's probability
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply Camshift to get the new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)

        # Draw it on image 
        # We use polylines to represent Adaptive box 
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, 255,2)
        
        cv2.imshow('Camshift Tracking', img2)
        
        if cv2.waitKey(1) == 13: #13 is the Enter Key
            break

    else:
        break

cv2.destroyAllWindows()
cap.release()

این کد پیاده‌سازی الگوریتم Camshift برای ردیابی شیء در تصویر دریافتی از وب‌کم است. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- وب‌کم با استفاده از `cv2.VideoCapture(0)` فعال می‌شود.
- اولین فریم از وب‌کم خوانده می‌شود تا ناحیه اولیه ردیابی انتخاب شود.
- مختصات و ابعاد پنجره ردیابی (`track_window`) تعیین می‌شود و ناحیه مورد نظر (ROI) از تصویر بریده می‌شود.
- ناحیه انتخابی به فضای رنگی HSV تبدیل می‌شود تا فیلتر رنگ راحت‌تر انجام شود.
- یک ماسک برای محدوده رنگ بنفش ایجاد می‌شود تا فقط پیکسل‌های این رنگ در ناحیه مورد نظر باقی بمانند.
- هیستوگرام رنگی ناحیه انتخابی محاسبه و نرمال‌سازی می‌شود تا برای ردیابی استفاده شود.
- معیار توقف الگوریتم Camshift تعیین می‌شود (۱۰ تکرار یا جابجایی کمتر از ۱ پیکسل).
- در یک حلقه بی‌نهایت:
    - هر بار یک فریم جدید از وب‌کم خوانده می‌شود.
    - تصویر به فضای HSV تبدیل می‌شود.
    - تصویر back projection هیستوگرام محاسبه می‌شود تا احتمال حضور شیء در هر نقطه مشخص شود.
    - الگوریتم Camshift روی تصویر back projection اجرا می‌شود تا موقعیت و اندازه جدید پنجره ردیابی به دست آید.
    - یک چندضلعی (مستطیل تطبیقی) روی شیء ردیابی‌شده رسم و نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
- در پایان، تمام پنجره‌ها بسته شده و وب‌کم آزاد می‌شود.

این کد برای ردیابی اشیاء رنگی (در اینجا بنفش) در تصویر ویدیویی کاربرد دارد و با استفاده از الگوریتم Camshift موقعیت و اندازه پنجره ردیابی را در هر فریم به‌روزرسانی می‌کند.


## When and how to use Meanshift or Camshift (چه زمانی و چگونه از Meanshift یا Camshift استفاده کنیم)

انتخاب بین الگوریتم Meanshift و Camshift بستگی به ویژگی‌های شیء مورد ردیابی و شرایط صحنه دارد:

- **Meanshift** زمانی مناسب است که:
    - اندازه و شکل شیء مورد ردیابی در طول زمان تقریباً ثابت بماند.
    - اطلاعات اولیه دقیقی از موقعیت و ابعاد شیء دارید.
    - نیازی به تغییر اندازه یا چرخش پنجره ردیابی ندارید.

- **Camshift** زمانی مناسب است که:
    - اندازه، شکل یا جهت شیء در طول زمان تغییر می‌کند (مثلاً شیء به دوربین نزدیک یا از آن دور می‌شود).
    - می‌خواهید پنجره ردیابی به صورت خودکار با تغییرات شیء تطبیق پیدا کند.
    - انعطاف‌پذیری بیشتری برای ردیابی اشیاء متحرک و تغییرپذیر نیاز دارید.

**نکته:** تعیین صحیح موقعیت و اندازه اولیه پنجره ردیابی بسیار مهم است. اگر این مقدار به‌درستی انتخاب نشود، هر دو الگوریتم ممکن است به جای شیء اصلی، یک ناحیه اشتباه را دنبال کنند یا در یک کمینه محلی گیر کنند.

## Optical Flow (جریان نوری)
به‌دنبال به‌دست آوردن الگوی حرکت ظاهری اشیاء در یک تصویر بین دو فریم متوالی است.
توزیع سرعت‌های ظاهری اشیاء در یک تصویر را نمایش می‌دهد.
## optical flow implementations in Opencv (پیاده‌سازی‌های جریان نوری در OpenCV)
در OpenCV دو پیاده‌سازی برای optical flow وجود دارد:
- روش تفاضلی Lucas-Kanade – برخی نقاط کلیدی را در ویدیو دنبال می‌کند و برای ویژگی‌های گوشه‌دار مناسب است (مثلاً ردیابی خودروها از نمای پهپاد).
- optical flow چگال (dense optical flow) – کندتر است اما جریان نوری را برای همه نقاط فریم محاسبه می‌کند، برخلاف Lucas-Kanade که فقط از ویژگی‌های گوشه‌ای (مجموعه داده پراکنده) استفاده می‌کند. در این روش، رنگ‌ها برای نمایش حرکت به‌کار می‌روند؛ Hue جهت حرکت و Value (روشنایی/شدت) سرعت را نشان می‌دهد.


In [ ]:
import numpy as np
import cv2

# Load video stream
cap = cv2.VideoCapture('images/walking.avi')

# Set parameters for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Set parameters for lucas kanade optical flow
lucas_kanade_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
# Used to create our trails for object movement in the image 
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Find inital corner locations
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(prev_frame)

while(1):
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    new_corners, status, errors = cv2.calcOpticalFlowPyrLK(prev_gray, 
                                                           frame_gray, 
                                                           prev_corners, 
                                                           None, 
                                                           **lucas_kanade_params)

    # Select and store good points
    good_new = new_corners[status==1]
    good_old = prev_corners[status==1]

    # Draw the tracks
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a,b), 5, color[i].tolist(),-1)
        
    img = cv2.add(frame,mask)

    # Show Optical Flow
    cv2.imshow('Optical Flow - Lucas-Kanade',img)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

    # Now update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prev_corners = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

این کد پیاده‌سازی جریان نوری Lucas-Kanade برای ردیابی حرکت نقاط کلیدی در یک ویدیو است. مراحل کار به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`numpy` و `cv2`) وارد می‌شوند.
- ویدیوی `'images/walking.avi'` بارگذاری می‌شود.
- پارامترهای مورد نیاز برای شناسایی گوشه‌ها با الگوریتم Shi-Tomasi تعیین می‌شود.
- پارامترهای الگوریتم Lucas-Kanade برای محاسبه جریان نوری تنظیم می‌شود.
- ۱۰۰ رنگ تصادفی برای رسم مسیر حرکت نقاط تولید می‌شود.
- اولین فریم ویدیو خوانده شده و به تصویر خاکستری تبدیل می‌شود.
- نقاط گوشه‌ای (ویژگی‌های مناسب برای ردیابی) در اولین فریم شناسایی می‌شوند.
- یک تصویر ماسک (برای رسم خطوط حرکت) با همان اندازه فریم ساخته می‌شود.
- در یک حلقه:
    - هر بار یک فریم جدید خوانده و به خاکستری تبدیل می‌شود.
    - جریان نوری بین فریم قبلی و فعلی با استفاده از `cv2.calcOpticalFlowPyrLK` محاسبه می‌شود.
    - نقاط خوب جدید و قدیمی انتخاب می‌شوند (نقاطی که با موفقیت ردیابی شده‌اند).
    - برای هر نقطه، یک خط بین موقعیت قبلی و جدید رسم می‌شود و یک دایره روی نقطه جدید کشیده می‌شود.
    - تصویر نهایی با جمع کردن ماسک و فریم فعلی ساخته می‌شود و نمایش داده می‌شود.
    - اگر کلید Enter فشرده شود، حلقه متوقف می‌شود.
    - فریم و نقاط فعلی برای تکرار بعدی به‌روزرسانی می‌شوند.
- در پایان، تمام پنجره‌ها بسته شده و ویدیو آزاد می‌شود.

این کد برای نمایش مسیر حرکت نقاط کلیدی در ویدیو و تحلیل حرکت اشیا کاربرد دارد.
